# Simple Test of Reciprocity of 3 Sources and 3 Recievers in Groningen

### Step 0

Load packages

In [ ]:
#load all packages
import datetime
import pickle
import copy
import os

from pathlib import Path

import numpy as np
import pandas as pd
import pyvista as pv
import matplotlib.pyplot as plt 

from sys import argv

from matplotlib.colors import Normalize
from pyaspect.model.gridmod3d import gridmod3d as gm
from pyaspect.model.bbox import bbox as bb
from pyaspect.model.gm3d_utils import *
from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.headers import *
#from pyaspect.specfemio.write import *
from pyaspect.specfemio.read import *
from pyaspect.specfemio.utils import *

import pyaspect.events.gevents as gevents
import pyaspect.events.gstations as gstations
from pyaspect.events.munge.knmi import correct_station_depths as csd_f
import pyaspect.events.mtensors as mtensors
from obspy.imaging.beachball import beach
from obspy import UTCDateTime
import shapefile as sf

### Step 1 

Extract the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_in_dir}
!ls data/groningen

### Step 6 

Decompress the ndarray of the sliced, subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# set filename then used it to decompress model
ifqn = f'{data_out_dir}/vsliced_subsmp_smth_nam_2017_vp_vs_rho_Q_model_dx100_dy100_dz100_maxdepth5850_sig250.npz'
vslice_gm3d, other_pars = decompress_gm3d_from_file(ifqn)

print()
print('decompressed gridded model\n:',vslice_gm3d) 
print()
print('other parameters:\n',other_pars)
print()

# WARNING: this will unpack all other_pars, if you overwrite a variable of the samename as val(key), then you 
#          may not notice, and this may cause large headaches.  I use it because I am aware of it.
'''
for key in other_pars:
    locals()[key] = other_pars[key]  #this is more advanced python than I think is reasonable for most 
sig_meters = sig
''';

# another way to get these varibles is just use the accessor functions for the gridmod3d.  We need them later.
xmin = other_pars['xmin']
dx   = other_pars['dx']
nx   = other_pars['nx']
ymin = other_pars['ymin']
dy   = other_pars['dy']
ny   = other_pars['ny']
zmin = other_pars['zmin']
dz   = other_pars['dz']
nz   = other_pars['nz']
sig_meters = other_pars['sig']  # this variable is used later
print('sig_meters:',sig_meters)

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
nam_dims = list(vslice_gm3d.get_npoints())
nam_origin = [0,0,-vslice_gm3d.get_gorigin()[2]]
#nam_origin = list(vslice_gm3d.get_gorigin())
#nam_origin[2] *= -1
nam_origin = tuple(nam_origin)
nam_spacing = list(vslice_gm3d.get_deltas())
nam_spacing[2] *=-1
nam_spacing = tuple(nam_spacing)
print('nam_dims:',nam_dims)
print('nam_origin:',nam_origin)
print('nam_spacing:',nam_spacing)

# Edit the spatial reference
grid.dimensions = np.array(nam_dims) + 1
grid.origin = nam_origin  # The bottom left corner of the data set
grid.spacing = nam_spacing  # These are the cell sizes along each axis
nam_pvalues = vslice_gm3d.getNPArray()[0]
print('pvalues.shape:',nam_pvalues.shape)

# Add the data values to the cell data
grid.cell_arrays["values"] = nam_pvalues.flatten(order="F")  # Flatten the array!

# Now plot the grid!
cmap = plt.cm.jet
#grid.plot(show_edges=True,cmap=cmap)
grid.plot(cmap=cmap,opacity=1.0)


In [ ]:
slices = grid.slice_orthogonal()

#slices.plot(show_edges=True,cmap=cmap)
slices.plot(cmap=cmap)

## create random virtual source (to specfem stations, but using reciprocity -- sources)

In [ ]:
#coords = vslice_gm3d.getGlobalCoordsPointsXYZ()
coords = vslice_gm3d.getLocalCoordsPointsXYZ()
coords[:,2] = -coords[:,2]

xc = np.unique(coords.T[0,:])
yc = np.unique(coords.T[1,:])
zc = np.unique(coords.T[2,:])


#n_rand_p = 1000

n_rand_p = 3
np.random.seed(n_rand_p) #nothing special about using n_rand_p just want reproducible random

#stay away from the edges of the model for derivatives 
# and to avoid boundary effects
xy_pad = 500 

lrx = np.min(xc) + xy_pad
lry = np.min(yc) + xy_pad
lrz = -3400.0

hrx = np.max(xc) - xy_pad
hry = np.max(yc) - xy_pad
hrz = -2600.0

srx = hrx - lrx
sry = hry - lry
srz = hrz - lrz

r_xyz_list = []
for i in range(n_rand_p):
    rx = lrx + srx*np.random.rand()
    ry = lry + sry*np.random.rand()
    rz = lrz + srz*np.random.rand()
    r_xyz_list.append([rx,ry,rz])
    
r_xyz = np.array(r_xyz_list)
    

#r_xyz = np.vstack(np.meshgrid(rx,ry,rz)).reshape(3,-1).T
print('r_xyz:\n',r_xyz)


In [ ]:
pv_rpoints = pv.wrap(r_xyz)
p = pv.Plotter()
slices = grid.slice_orthogonal()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(slices,cmap=cmap,opacity=1)
p.add_mesh(grid,cmap=cmap,opacity=0.50)
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=1.0)

p.show()

## Make Moment Tensors and CMTSolutionHeaders for each tensor

In [ ]:
# this is the path to the project dir on the cluster
my_proj_dir = '/scratch/seismology/tcullison/test_mesh/FWD_Batch_Src_Test'

magnitude = np.pi
strike = [30,45,90] # just making three to test
dip = [30,30,60]
rake = [330,190,20]

l_mt = []
for i in range(len(strike)):
    l_mt.append(MomentTensor(mw=magnitude,strike=strike[i],dip=dip[i],rake=rake[i]))

assert len(l_mt) == len(r_xyz)

for mt in l_mt:
    print(mt)
    
l_cmt_src = []
for i in range(len(r_xyz)):
    cmt_h = CMTSolutionHeader(date=datetime.datetime.now(),
                              ename=f'Event-{str(i).zfill(4)}',
                              tshift=0.0,
                              hdur=0.0,
                              lat_yc=r_xyz[i,1],
                              lon_xc=r_xyz[i,0],
                              depth=-r_xyz[i,2],
                              mt=l_mt[i],
                              eid=i,
                              sid=0)
    l_cmt_src.append(cmt_h)
    
print()
for cmt in l_cmt_src:
    print(cmt)

## Make Corresponding "Virtual" Recievers (including cross membors for derivatives) for the CMT's

In [ ]:
m_delta = 50.0 # distance between cross stations for derivatives
assert m_delta < xy_pad #see cells above this is padding
#l_grp_vrecs = make_grouped_half_cross_reciprocal_station_headers_from_cmt_list(l_cmt_src,m_delta)
l_grp_vrecs = make_grouped_cross_reciprocal_station_headers_from_cmt_list(l_cmt_src,m_delta)

ig = 0
for grp in l_grp_vrecs:
    print(f'***** Group: {ig} *****\n')
    ir = 0
    for gvrec in grp:
        print(f'*** vrec: {ir} ***\n{gvrec}')
        ir += 1
    ig += 1

print(len(flatten_grouped_headers(l_grp_vrecs)))
    

## Plot Virtual Receiver Groups

In [ ]:
all_g_xyz = get_xyz_coords_from_station_list(flatten_grouped_headers(l_grp_vrecs))
all_g_xyz[:,2] *= -1 #pyview z-up positive and oposize sign of standard geophysics 
pv_all_points = pv.wrap(all_g_xyz)
p = pv.Plotter()
p.add_mesh(grid,cmap=cmap,opacity=0.5)
p.add_mesh(slices,cmap=cmap,opacity=1.0)
p.add_mesh(pv_all_points, render_points_as_spheres=True, point_size=5,opacity=1.0)
p.show()

## Get receiver/station coordinates created from a different notebook

In [ ]:
# unpickle the Bounding box (from a different notebook)

#ifqn  = data_out_dir + 'bbox_nvl' + str(int(nvl)) + '_nvb' + str(int(nvb))
#ifqn += '_xsft' + str(xshift) + '_ysft' + str(yshift) + '.pickle'
ifqn = data_out_dir + 'bbox_nvl152_nvb197_xsft4400_ysft19100.pickle'
f = open(ifqn, 'rb')
sgf_bbox = pickle.load(f)
f.close()
print()
print('Unpickled Bounding:\n',sgf_bbox)

In [ ]:
# unpickle the events if needed (again from a different notebook)
ifqn = data_out_dir + 'bbox_groning_events.pickle'
f = open(ifqn, 'rb')
bbox_events = pickle.load(f)
f.close()
print()
print('Unpickled Events:\n',bbox_events)

In [ ]:
#Read moment tensors
mt_in_file  = 'data/groningen/events/event_moments.csv' 
!ls {mt_in_file}
bbox_gf_mts = mtensors(mt_in_file)

# get event catalog of the events (ObsPy catalog)
bbox_event_cat = copy.deepcopy(bbox_events.getIncCatalog())

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
bbox_gf_mts.update_utcdatetime(bbox_event_cat)

'''
#for imt in range(len(bbox_gf_mts)):
#    print("Moment-Tensor %d:/n" %(imt),bbox_gf_mts[imt])
'''

# Create a dictionary that maps moment tensors to events
bbox_emap,bbox_mt_cat,bbox_mts = bbox_gf_mts.get_intersect_map_events_mts(bbox_event_cat)
bbox_e2mt_keys = bbox_emap.keys()

# Print a comparison of events to moment tensors
for key in bbox_e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,bbox_mt_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,bbox_emap[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,bbox_mt_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,bbox_emap[key]['ML']))
    print()

#replace moment-tensors with only those that intersect with the events in the BoundingBox
bbox_gf_mts.replace_moment_tensors_from_map(bbox_emap)
    
# add mt_catalog to bbox_events
bbox_events.mergeMomentTensorsCatalog(bbox_mt_cat)
merged_bbox_event_cat = bbox_events.getIncCatalog()
print('bbox_event_cat:\n', bbox_event_cat)
print()
print('merged_bbox_event_cat:\n', merged_bbox_event_cat)
print()
print('bbox_mt_cat:\n', bbox_mt_cat)
print()
print('bbox_mt_df:\n', bbox_gf_mts)

In [ ]:
ifqn = data_out_dir + 'bbox_groning_stations.pickle'

print('Unpickling Station Traces')
f = open(ifqn, 'rb')
bbox_straces = pickle.load(f)
f.close()

print('Stations:\n',type(bbox_straces))

In [ ]:
# read shapefiles
shape_in_files  = 'data/groningen/shapefile/Groningen_field' 

gf_shape = sf.Reader(shape_in_files)
print('Groningen Field shape:',gf_shape)

#get coordinates for the Shape-File
s = gf_shape.shape(0)
shape_xy = np.asarray(s.points)

In [ ]:
# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
#from gnam.events.munge.knmi import correct_station_depths as csd_f
bbox_straces.correct_stations(csd_f)

bbox_bb_diam = 1500  #size of the beachball for plotting. I had to play with this parameter
bbox_cmt_bballs = bbox_gf_mts.get_cmt_beachballs(diam=bbox_bb_diam,fc='black')

bbox_mt_coords = bbox_events.getIncCoords()

#get event and borhole keys used for indexing
ekeys = bbox_straces.getEventKeys()
bkeys = bbox_straces.getBoreholeKeys()

#Plot seuence of events with stations 
#for ie in ekeys:
for i in range(1):
    ie = ekeys[i]
    # coordinates for stations that are in the bounding box
    xy3 = bbox_straces.getIncStationCoords(ie,bkeys[0]) #station code G##3
    xy4 = bbox_straces.getIncStationCoords(ie,bkeys[1]) #station code G##4
    
    # coordinates for stations that are G-stations but outside the bounding box
    ex_xy3 = bbox_straces.getExcStationCoords(ie,bkeys[0]) #station code G##3
    ex_xy4 = bbox_straces.getExcStationCoords(ie,bkeys[1]) #station code G##4
    
    # coordinates for stations that are inside the bounding box but there is no data
    er_xy3 = bbox_straces.getErrStationCoords(ie,bkeys[0]) #station code G##3
    er_xy4 = bbox_straces.getErrStationCoords(ie,bkeys[1]) #station code G##4

    fig, ax = plt.subplots(1,figsize=(8,8))
    fig.gca().set_aspect('equal', adjustable='box')
    
    #Groningen Field Shape-File
    ax.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)
    
    #Bounding Box
    ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='green',zorder=1)
    
    #Events (reuse event coordinates from cell further above)
    ax.scatter(bbox_mt_coords[ie,0],bbox_mt_coords[ie,1],s=90,c='red',marker='*',zorder=5)
    beach = bbox_cmt_bballs[ie]  #this creates a plot collection for the beachball points
    beach.set_zorder(3)
    ax.add_collection(beach)
    
    #Included stations
    ax.scatter(xy3[:,0],xy3[:,1],s=50,c='blue',marker='v',zorder=3)
    ax.scatter(xy4[:,0],xy4[:,1],s=100,c='gray',marker='o',zorder=2)
    
    #Excluded stations
    ax.scatter(ex_xy3[:,0],ex_xy3[:,1],s=80,c='lightgray',marker='1',zorder=4)
    ax.scatter(ex_xy4[:,0],ex_xy4[:,1],s=100,c='lightgray',marker='2',zorder=3)
    
    #Stations without data
    ax.scatter(er_xy3[:,0],er_xy3[:,1],s=50,c='yellow',marker='v',zorder=4)
    ax.scatter(er_xy4[:,0],er_xy4[:,1],s=100,c='gray',marker='o',zorder=3)
    
    origin_time = bbox_events[ie].origins[0].time
    mag = bbox_events[ie].magnitudes[0].mag
    title_str = 'Event-%d, Origin Time: %s, Magnitude: %1.2f' %(ie,str(origin_time),mag)
    ax.set_title(title_str)
    plt.show()
    


In [ ]:
fig1, ax1 = plt.subplots(1,figsize=(8,8))
fig1.gca().set_aspect('equal', adjustable='box')
    
#Groningen Field Shape-File
ax1.scatter(shape_xy[:,0],shape_xy[:,1],s=1,c='black',zorder=0)

#Bounding Box
ax1.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='green',zorder=1)

#Events (reuse event coordinates from cell further above)
ax1.scatter(bbox_mt_coords[ie,0],bbox_mt_coords[ie,1],s=90,c='red',marker='*',zorder=5)
    
all_xy4 = np.concatenate((xy4,er_xy4),axis=0)
#all_xy4 = xy4

ax1.scatter(all_xy4[:,0],all_xy4[:,1],s=100,c='gray',marker='o',zorder=2)
ax1.scatter(er_xy4[:,0],er_xy4[:,1],s=100,c='yellow',marker='x',zorder=2)

title_str = 'Event-%d, Origin Time: %s, Magnitude: %1.2f' %(ie,str(origin_time),mag)
ax1.set_title(title_str)
plt.show()

## Make random virtual sources

In [ ]:
coords = vslice_gm3d.getLocalCoordsPointsXY()

x_orig = vslice_gm3d.get_gorigin()[0]
y_orig = vslice_gm3d.get_gorigin()[1]

clip_xy = all_xy4[9:13]
print(clip_xy)

s_xyz = np.zeros((len(clip_xy),3))
s_xyz[:,0] = clip_xy[:,0] - x_orig
s_xyz[:,1] = clip_xy[:,1] - y_orig
s_xyz[:,2] = -200

print(s_xyz)

## Plot virtual sources (red) with virtual receivers (white)

In [ ]:
pv_spoints = pv.wrap(s_xyz)
p = pv.Plotter()
#p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
#p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.add_mesh(all_g_xyz, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## Make StationHeaders (real recievers) 

In [ ]:
l_real_recs = []
for i in range(len(s_xyz)):
    
    tr_bname = 'tr'
    new_r = StationHeader(name=tr_bname,
                          network='NL', #FIXME
                          lon_xc=s_xyz[i,0],
                          lat_yc=s_xyz[i,1],
                          depth=-s_xyz[i,2], #specfem z-down is positive
                          elevation=0.0,
                          trid=i)
    l_real_recs.append(new_r)
    
for rec in l_real_recs:
    print(rec)


## Make ForceSolutionHeaders for the above virtual sources (including force-triplets for calculation derivatives)

In [ ]:
l_grp_vsrcs = make_grouped_reciprocal_force_solution_triplet_headers_from_rec_list(l_real_recs)

## Make replicates of each virtual receiver list: one for each force-triplet

In [ ]:
l_grp_vrecs_by_vsrcs = make_replicated_reciprocal_station_headers_from_src_triplet_list(l_grp_vsrcs,
                                                                                          l_grp_vrecs)

## Plot virtual sources (red) and virtual receivers (white) FROM headers

In [ ]:
grp_s_xyz = get_unique_xyz_coords_from_solution_list(flatten_grouped_headers(l_grp_vsrcs))
grp_s_xyz[:,2] *= -1 #pyvista z-up is positive

flat_recs = flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs))
grp_r_xyz = get_unique_xyz_coords_from_station_list(flat_recs)
grp_r_xyz[:,2] *= -1 #pyvista z-up is positive

print(len(grp_s_xyz))
print(len(grp_r_xyz))

pv_spoints = pv.wrap(grp_s_xyz)
pv_rpoints = pv.wrap(grp_r_xyz)

p = pv.Plotter()
p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=8,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=5,opacity=0.5)
p.show()

## Make replicates of each "real" receiver list: for each CMT source

In [ ]:
l_grp_recs_by_srcs = make_replicated_station_headers_from_src_list(l_cmt_src,l_real_recs)


for i in range(len(l_cmt_src)):
    print(f'***** SRC Records for Source: {i} *****\n')
    for j in range(len(l_real_recs)):
        print(f'*** REC Header for Receiver: {j} ***\n{l_grp_recs_by_srcs[i][j]}')
    

## Plot "real" sources (red) and virtual receivers (white) FROM headers

In [ ]:
grp_s_xyz = get_unique_xyz_coords_from_solution_list(l_cmt_src)
grp_s_xyz[:,2] *= -1 #pyvista z-up is positive

flat_recs = flatten_grouped_headers(l_grp_recs_by_srcs) #real!
grp_r_xyz = get_unique_xyz_coords_from_station_list(flat_recs)
grp_r_xyz[:,2] *= -1 #pyvista z-up is positive

print(len(grp_s_xyz))
print(len(grp_r_xyz))

pv_spoints = pv.wrap(grp_s_xyz)
pv_rpoints = pv.wrap(grp_r_xyz)

p = pv.Plotter()
p.add_mesh(slices,cmap=cmap,opacity=0.50)
p.add_mesh(grid,cmap=cmap,opacity=0.3)
p.add_mesh(pv_spoints, render_points_as_spheres=True, point_size=12,opacity=1,color='red')
p.add_mesh(pv_rpoints, render_points_as_spheres=True, point_size=8,opacity=0.5)
p.show()

## Make reciprical RecordHeader

In [ ]:
print(len(flatten_grouped_headers(l_grp_vsrcs.copy())))
print(len(flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs.copy()))))
print('nrec_per_src*nsrc:',21*12)

l_flat_vsrcs = flatten_grouped_headers(l_grp_vsrcs)
l_flat_vrecs = flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs))

vrecord_h = RecordHeader(name='Reciprocal-Record',solutions_h=l_flat_vsrcs,stations_h=l_flat_vrecs)

'''
#print(vrecord_h.get_event_nsolutions(1))
#print(vrecord_h)

idx = pd.IndexSlice
slice_rec_h = vrecord_h.copy()
slice_rec_h.solutions_df.reset_index(inplace=True)
#slice_rec_h.stations_df.reset_index(inplace=True)
slice_rec_h.stations_df
rec_df = slice_rec_h.stations_df
#print(f'rec_df = {rec_df}')
for index, src in slice_rec_h.solutions_df.iterrows():
    print(f'**** src.sid = {src.sid} ****************\n')
    print(f'**** src.eid = {src.eid} ****************\n')
    #print(rec_df[rec_df['proj_id'] == src.proj_id])
    #print(f'index = {index}')
    #new_rec_df = rec_df[(rec_df['proj_id'] == src.proj_id) & (rec_df['eid'] == src.eid) & (rec_df['sid'] == src.sid)]
    for index,rec in rec_df.loc[idx[src.proj_id,src.eid,src.sid],:].reset_index().iterrows():
        print(rec)
#slice_rec_h.stations_df.loc[idx[0,0,:,:],'data_fqdn'] = '/somewhere/over/the/rainbow'
#print(slice_rec_h.stations_df.loc[idx[0,0,:,:],'data_fqdn'])
''';

print()
svr = vrecord_h[0,:,:,:]
print('svr:',svr)
svr.solutions_df['proj_id'] = 1
svr.stations_df['proj_id'] = 1
print()
print('proj svr:',svr)

#print('index:',svr.index)
#print('shape:',svr.index.shape)

'''
svr_idx_names = svr.index.names
print('names:\n',svr_idx_names)
print()
print('orig:\n',svr)
svr.reset_index(inplace=True)
print()
print('reset:\n',svr)
svr['proj_id'] = 1
print()
print('new proj_id:\n',svr)
svr.set_index(svr_idx_names)
print()
print('new idx:\n',svr)
''';

assert False

## Redo Make Project Code

In [ ]:
from pyaspect.specfemio.write import write_solution
from pyaspect.specfemio.write import write_stations
from pyaspect.specfemio.write import _write_header

def write_record(rdir_fqp,
                 record_h,
                 fname='event_record',
                 write_spec_files=True,
                 write_record_h=True,
                 write_h=False,
                 auto_name=False,
                 auto_network=False):

    data_fqp   = _join_path_fname(rdir_fqp,'DATA')
    record_fqp = _get_header_path(data_fqp,fname)


    # write SPECFEM solution files and headers
    if write_spec_files:
        srcs_df = record_h.solutions_df.copy()
        srcs_df.reset_index(inplace=True)

        recs_df = record_h.stations_df.copy()
        recs_df.reset_index(inplace=True)

        i = 0
        for sindex,src in srcs_df.iterrows():

            mk_sym_link = False
            if i == 0: #write SOLUTION symlink
                mk_sym_link = True
            SolHeaderCls = record_h._get_header_class(is_stations=False)
            solution_h = SolHeaderCls.from_series(src)
            write_solution(data_fqp,solution_h,postfix=f'sid{src.sid}',write_h=write_h,mk_sym_link=mk_sym_link)
            i += 1


            #get related stations
            slice_recs_df = recs_df[(rec_df['proj_id'] == src.proj_id)]
            print(f'slice_recs_df:\n{slice_recs_df}')
            '''
            slice_recs_df = recs_df[(rec_df['proj_id'] == src.proj_id) &
                                    (rec_df['eid'] == src.eid) &
                                    (rec_df['sid'] == src.sid)]

            l_stations = []
            for rindex,rec in slice_recs_df.iterrows():

                #TODO: could make '/SYN' dir variable dynamic
                #      also with out knowing specfem DT, can't
                #      finish the name of the station data (trace)
                data_fname = station_auto_data_fname_id(rec)
                rec.data_fqdn = _join_path_fname(edir_fqp,f'/SYN/{data_fname}')

                StatHeaderCls = record_h._get_header_class(is_stations=True)
                l_stations.append(SolHeaderCls.from_series(rec))


            r_fname = f'STATIONS.sid{src.sid}'
            write_stations(data_fqp,
                           l_stations,
                           fname=r_fname,
                           write_h=write_h,
                           auto_name=auto_name,
                           auto_network=auto_network,
                           mk_sym_link=mk_sym_link)
            ''';

    '''
    # write record header
    if write_record_h:
        _write_header(record_fqp,record_h)
    ''';


In [ ]:
from pyaspect.specfemio.utils import make_record_headers
from pyaspect.specfemio.utils import _mk_symlink
from pyaspect.specfemio.utils import _copy_recursive_dir
from pyaspect.specfemio.utils import _join_path_fname
from pyaspect.specfemio.utils import _get_header_path
from pyaspect.parfile import change_multiple_parameters_in_lines
from pyaspect.parfile import readlines
from pyaspect.parfile import writelines




MAX_SPEC_SRC = int(9999) # see SPECFEM3D_Cartesian manual

# list of directories need for every event
common_dir_struct = {'DATA': {},
                     'OUTPUT_FILES' : {'DATABASES_MPI':{}},
                     'SYN': {},
                     'FILT_SYN': {} }

# extra common dirs for fwi
common_fwi_dir_struct = {'SEM': {},
                         'OBS': {},
                         'FILT_OBS': {} }

# list of directories only needed for the primary run0001 dir
primary_dir_struct= {'INPUT_GRADIENT': {},
                    'INPUT_KERNELS': {},
                    'INPUT_MODEL': {},
                    'OUTPUT_MODEL': {},
                    'OUTPUT_SUM': {},
                    'SMOOTH': {},
                    'COMBINE': {},
                    'topo': {} }


def set_proj_id(record_h, proj_ival):
    rec_df = record_h.stations_df
    src_df = record_h.solutions_df
    rec_names = rec_df.index.names
    src_names = src_df.index.names
    rec_df.reset_index(inplace=True)
    src_df.reset_index(inplace=True)
    rec_df['proj_id'] = 1
    src_df['proj_id'] = 1
    rec_df.set_index(rec_names)
    src_df.set_index(src_names)
    
    

def _make_dirs(fqdn,access_rights=0o755):
    if os.path.isdir(fqdn):
        raise OSError(f'The directory {fqdn} has already been created')
    try:
        os.makedirs(fqdn, access_rights)
    except OSError:
        print(f'Creation of the directory {fqdn} failed')
        return OSError
    
    
def _recursive_proj_dirs(dl,pdir,access_rights=0o755):

    if len(dl.keys()) == 0:
        return
    else:
        for dl_key in dl.keys():
            new_dir = os.path.join(pdir, dl_key)
            _make_dirs(new_dir,access_rights=0o755)
            _recursive_proj_dirs(dl[dl_key],new_dir)


def _make_proj_dir(proj_root_fqp,
                   proj_base_name,
                   pyutils_fqp=None,
                   script_fqp=None):

        projdir_fqp = os.path.join(proj_root_fqp, proj_base_name)
        print(f'projdir_fqp: {projdir_fqp}')
        _make_dirs(projdir_fqp)
        
        # create project level symlinks 
        if pyutils_fqp != None:
            lname = 'pyutils'
            src = pyutils_fqp
            dst = os.path.join(projdir_fqp, lname)
            _mk_symlink(src,dst)

        if script_fqp != None:
            lname = 'scriptutils'
            src = script_fqp
            dst = os.path.join(projdir_fqp, lname)
            _mk_symlink(src,dst)
        
        return projdir_fqp
            
def _make_run_dir(irdir,
                  projdir_fqp,
                  spec_bin_fqp,
                  spec_utils_fqp,
                  par_lines,
                  dir_struct,
                  record_h):

    rdir_name = 'run' + str(irdir+1).zfill(4)
    rundir_fqp = os.path.join(projdir_fqp, rdir_name)
    _make_dirs(rundir_fqp)

    # make sim links for each event dir 
    # (related to the computational node(s) filesytem
    lname = 'bin'
    src = spec_bin_fqp
    dst = os.path.join(rundir_fqp, lname)
    _mk_symlink(src,dst)

    lname = 'utils'
    src = spec_utils_fqp
    dst = os.path.join(rundir_fqp, lname)
    _mk_symlink(src,dst)

    # make subdirectorieds for each event
    _recursive_proj_dirs(common_dir_struct,rundir_fqp)

    #write Par_files in DATA dirs
    ddir_fqp = os.path.join(rundir_fqp, 'DATA')
    out_par_fqp  = os.path.join(ddir_fqp, 'Par_file')
    writelines(out_par_fqp,par_lines)
    
    #write Headers and Record
    write_record(rundir_fqp,
                 record_h,
                 fname='rdir_record',
                 write_spec_files=True,
                 write_record_h=True,
                 write_h=False,
                 auto_name=True,
                 auto_network=True)

    

def make_fwd_project_dir(proj_base_name,
                         proj_root_fqp,
                         parfile_fqp,
                         mesh_fqp,
                         spec_fqp,
                         pyutils_fqp,
                         script_fqp,
                         proj_record_h,
                         sub_proj_name=None,
                         batch_srcs=False,
                         copy_mesh=False,
                         max_event_rdirs=MAX_SPEC_SRC,
                         verbose=False):
    
    
    if not isinstance(proj_record_h,RecordHeader):
        raise TypeError('arg: \'record_h\' must be a RecordHeader type')

    if not isinstance(proj_base_name,str):
        raise TypeError('proj_base_name must be a str type')

    if not isinstance(proj_root_fqp,str):
        raise TypeError('proj_root_fqp must be a str type')

    if not isinstance(parfile_fqp,str):
        raise TypeError('parfile_fqp must be a str type')

    if not isinstance(mesh_fqp,str):
        raise TypeError('mesh_fqp must be a str type')

    if not isinstance(spec_fqp,str):
        raise TypeError('spec_fqp must be a str type')

    if not isinstance(pyutils_fqp,str):
        raise TypeError('pyutils_fqp must be a str type')

    if not isinstance(script_fqp,str):
        raise TypeError('script_fqp must be a str type')
        
    
    ########################################################################
    #
    # setup project structure parameters
    #
    ########################################################################
            
    nevents = 0        
    # get unique event id's and make sure the are the same
    s_nu_eid = list(src_df.index.get_level_values('eid').unique())
    r_nu_eid = list(rec_df.index.get_level_values('eid').unique())
    
    # checkt that stations and solutions agree on number of events
    if s_nu_eid != r_nu_eid:
        estr  = 'Number of events differ between Solutions and Stations'
        raise Exception(estr)
    else:
        nevents = len(s_nu_eid)
        
    
    nbatch = 0
    # get unique source id's and make sure the are the same
    s_nu_sid = list(src_df.index.get_level_values('sid').unique())
    r_nu_sid = list(rec_df.index.get_level_values('sid').unique())

    # checkt that stations and solutions agree on nbatch
    if s_nu_sid != r_nu_sid:
        estr  = 'batch_srcs was specified, but unique source id\'s'
        estr += ' differ between Solutions and Stations'
        raise Exception(estr)
    else:
        nbatch = 1
        if batch_srcs:
            nbatch = len(s_nu_sid)
            
       
    # calculate number of rundirs (events) per subproject/project
    l_nrundirs = [nevents]
    if not batch_srcs:
        l_nrundirs[0] = nevents*len(s_nu_sid)
        
    # calculate if project or subprojects and addjust rundirs
    if max_event_rdirs < l_nrundirs[0]:
        old_nrundirs = l_nrundirs[0]
        ndiv = l_nrundirs[0]//max_event_rdirs
        l_nrundirs[0] = max_event_rdirs
        for i in range(1,ndiv):
            l_nrundirs.append(max_event_rdirs)
        rem_rundirs = old_nrundirs%max_event_rdirs 
        if rem_rundirs != 0:
            l_nrundirs.append(rem_rundirs)
            
    # actual number of subproject dirs
    nprojdirs = len(l_nrundirs)
    
    # info
    if verbose:
        print(f'nevents: {nevents}')
        print(f'nbatch: {nbatch}')
        print(f'l_nrundirs: {l_nrundirs}')
        print(f'total_rundirs: {sum(l_nrundirs)}')
        print(f'nprojdirs: {nprojdirs}')
        
        
    # setup paths to specfem binarys and utils/tools
    spec_bin_fqp   = os.path.join(spec_fqp, 'bin')
    spec_utils_fqp   = os.path.join(spec_fqp, 'utils')
    
    
    
    # Read input Par_file stub 
    par_lines = readlines(parfile_fqp)
    
    # Setup output Par_files based on user input Par_file stub
    par_keys = ['SIMULATION_TYPE','SAVE_FORWARD','MODEL','SAVE_MESH_FILES','USE_BINARY_FOR_SEISMOGRAMS']
    keys_vals_dict = dict(zip(par_keys,[1,False,'gll',False,True]))
    par_lines = change_multiple_parameters_in_lines(par_lines,keys_vals_dict)
    
    
    ########################################################################
    #
    # If more than one proj_dir is needed then make a main proj_dir
    # for the sub_proj_dirs.  
    #
    ########################################################################
    
    #Setup sub_dir pars here
    if sub_proj_name == None:
        sub_proj_name = 'Sub_' + proj_base_name
        
    sub_proj_root_fqp = proj_root_fqp
    sub_pdir_name = proj_base_name
    
    
    # make the main_dir if needed
    if 1 < nprojdirs:
        sub_proj_root_fqp = _make_proj_dir(proj_root_fqp,
                                           proj_base_name)
        
        
    if verbose: print(f'sub_proj_root_fqp: {sub_proj_root_fqp}')
        
    
    ie = 0
    for ipdir in range(nprojdirs):
        
        # Make subdir or main project dir depending on number of proj_dirs
        sub_projdir_fqp = sub_proj_root_fqp
        
        if nprojdirs != 1:
            sub_pdir_name = sub_proj_name + '_' + str(ipdir+1).zfill(4)
            
        sub_projdir_fqp = _make_proj_dir(sub_projdir_fqp,
                                         sub_pdir_name,
                                         pyutils_fqp=pyutils_fqp,
                                         script_fqp=script_fqp,)
        
        
        ####################################################################
        #
        # Make all the run dirs per proj/sub_proj dirs
        #
        ####################################################################
        isrc = 0
        for irdir in range(l_nrundirs[ipdir]):
            
            rdir_record_h = proj_record_h[ie,isrc:isrc+nbatch-1,:,:]
            
            rdir_record_h.solutions_df['proj_id'] = ipdir
            rdir_record_h.stations_df['proj_id']  = ipdir
 
            _make_run_dir(irdir,
                          sub_projdir_fqp,
                          spec_bin_fqp,
                          spec_utils_fqp,
                          par_lines,
                          common_dir_struct,
                          rdir_record_h)
            




In [ ]:
test_proj_name = 'TestNewMKProject'
test_proj_root_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects/NewMKProj')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_mesh_fqp = '/scratch/seismology/tcullison/test_mesh/MESH-default_batch_force_src'
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'

#print(len(flatten_grouped_headers(l_grp_vsrcs.copy())))
#print(len(flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs.copy()))))
#print('nrec_per_src*nsrc:',21*12)

l_flat_vsrcs = flatten_grouped_headers(l_grp_vsrcs)
l_flat_vrecs = flatten_grouped_headers(flatten_grouped_headers(l_grp_vrecs_by_vsrcs))

vrecord_h = RecordHeader(name='Reciprocal-Record',solutions_h=l_flat_vsrcs,stations_h=l_flat_vrecs)

#print(vrecord_h)
src_df = vrecord_h.solutions_df
rec_df = vrecord_h.stations_df

s_nsid = list(src_df.index.get_level_values('sid').unique())
r_nsid = list(rec_df.index.get_level_values('sid').unique())

#print(f'Are Same: {s_nsid == r_nsid}')

d = {'one':1, 'two':2, 'three': 3}
k = list(d.keys())

isin = 'three' in k
#print(f'is in: {isin}')

test_proj_record_h = vrecord_h.copy()

make_fwd_project_dir(test_proj_name,
                     test_proj_root_fqp,
                     test_parfile_fqp,
                     test_mesh_fqp,
                     test_spec_fqp,
                     test_pyutils_fqp,
                     test_script_fqp,
                     test_proj_record_h,
                     batch_srcs=False,
                     verbose=True,
                     max_event_rdirs=5)
                     #max_event_rdirs=MAX_SPEC_SRC)
        

print()
print('ls:')
!ls {test_proj_root_fqp}
print('ls:')
!ls {test_proj_root_fqp}/*/*
'''
print('rm:')
!rm -rf {test_proj_root_fqp}/*
print('ls:')
!ls {test_proj_root_fqp}
''';

## Make reciprocal project

In [ ]:
assert False
from pyaspect.specfemio.utils import _join_path_fname
test_proj_name = 'ReciprocalTestProject'
test_proj_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_mesh_fqp = '/scratch/seismology/tcullison/test_mesh/MESH-default_batch_force_src'
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'

from pyaspect.project import make_project
#make_records(l_src=l_grp_solutions_h,l_rec=src_grouped_stations)
make_project(test_proj_name,
             test_proj_fqp,
             test_parfile_fqp,
             test_mesh_fqp,
             test_spec_fqp,
             test_pyutils_fqp,
             test_script_fqp,
             l_grp_vsrcs,
             l_grp_vrecs_by_vsrcs,
             copy_mesh=False)

!ls -ltrh {os.path.join(test_proj_fqp,test_proj_name)}

## Make "real" project

In [ ]:
from pyaspect.specfemio.utils import _join_path_fname
test_proj_name = 'ForwardTestProject'
test_proj_fqp =  os.path.join(data_out_dir, 'tmp/TestProjects')
test_parfile_fqp =  os.path.join(data_out_dir, 'Par_file')
test_mesh_fqp = '/scratch/seismology/tcullison/test_mesh/MESH-default_batch_force_src'
test_spec_fqp = '/quanta1/home/tcullison/DevGPU_specfem3d'
test_pyutils_fqp = '/quanta1/home/tcullison/myscripts/python/specfem/pyutils'
test_script_fqp = '/quanta1/home/tcullison/myscripts/specfem'

from pyaspect.project import make_project
#make_records(l_src=l_grp_solutions_h,l_rec=src_grouped_stations)
make_project(test_proj_name,
             test_proj_fqp,
             test_parfile_fqp,
             test_mesh_fqp,
             test_spec_fqp,
             test_pyutils_fqp,
             test_script_fqp,
             l_cmt_src,
             l_grp_recs_by_srcs,
             copy_mesh=False)

!ls -ltrh {os.path.join(test_proj_fqp,test_proj_name)}

In [ ]:
assert False

In [ ]:
from pyaspect.parfile import change_parameter
from pyaspect.parfile import change_multiple_parameters

parfile_fqp = os.path.join(data_out_dir,'Par_file')
out_parfile_fqp = os.path.join(data_out_dir,'New_Par_file')

with open(parfile_fqp, 'r') as f:
    lines = f.readlines()

il = 0
for line in lines:
    if il == 313:
        break
    print(line)
    il += 1
'''
''';
    
#change_parameter(parfile_fqp,lines,'SAVE_FORWARD',True,out_parfile_fqp=out_parfile_fqp)
par_keys = ['SIMULATION_TYPE','SAVE_FORWARD','NSTEP','DT','MODEL','MIN_ATTENUATION_PERIOD','LOCAL_PATH','SAVE_MESH_FILES']
par_vals = [4,True,8192,0.002,'default',666666666,'./BOOGER/in_my_nose',True]
s_keys_vals = set(zip(par_keys,par_vals))
print(f's_keys_vals:\n{s_keys_vals}')
print(f'd_keys_vals:\n{[(k,v) for k,v in dict(zip(par_keys,par_vals)).items()]}')

change_multiple_parameters(parfile_fqp,s_keys_vals,out_parfile_fqp=out_parfile_fqp)
    

print('\n\n----------------------- changed -------------------------')

with open(out_parfile_fqp, 'r') as f:
    lines = f.readlines()
    
il = 0
for line in lines:
    if il == 313:
        break
    print(line)
    il += 1
'''
''';


In [ ]:
!diff {parfile_fqp} {out_parfile_fqp}

In [ ]:
import json


out_parfile_json = os.path.join(data_out_dir,'Par_file.json')
with open(out_parfile_json, 'w') as f:
    json.dump(lines,f)

In [ ]:
parfile_fqp = os.path.join(data_out_dir,'Par_file')
with open(parfile_fqp, 'r') as f:
    lines = f.readlines()
    
parfile_pickle = pickle.dumps(lines, 0)
parfile_pickle_str = parfile_pickle.decode('ascii')
#parfile_pickle_str = str(parfile_pickle)

#print(f'Parfile Type:\n{type(parfile_pickle)}')
print(f'Parfile String:\n{type(parfile_pickle_str)}')
print(f'Parfile String:\n{parfile_pickle_str.__repr__()}')

new_par_bytes = parfile_pickle_str.encode('ascii')
new_lines = pickle.loads(new_par_bytes)

'''
print('---------- The Winding Road -----------------')
for line in new_lines:
    print(line)
''';




In [ ]:
test_parfile_str = """(lp0\nV#-----------------------------------------------------------\\u000a\np1\naV#\\u000a\np2\naV# Simulation input parameters\\u000a\np3\naV#\\u000a\np4\naV#-----------------------------------------------------------\\u000a\np5\naV\\u000a\np6\naV# forward or adjoint simulation\\u000a\np7\naV# 1 = forward, 2 = adjoint, 3 = both simultaneously\\u000a\np8\naVSIMULATION_TYPE                 = 1\\u000a\np9\naV# 0 = earthquake simulation,  1/2/3 = three steps in noise simulation\\u000a\np10\naVNOISE_TOMOGRAPHY                = 0\\u000a\np11\naVSAVE_FORWARD                    = .false.\\u000a\np12\nag6\naV# solve a full FWI inverse problem from a single calling program with no I/Os, storing everything in memory,\\u000a\np13\naV# or run a classical forward or adjoint problem only and save the seismograms and/or sensitivity kernels to disk (with costlier I/Os)\\u000a\np14\naVINVERSE_FWI_FULL_PROBLEM        = .false.\\u000a\np15\nag6\naV# UTM projection parameters\\u000a\np16\naV# Use a negative zone number for the Southern hemisphere:\\u000a\np17\naV# The Northern hemisphere corresponds to zones +1 to +60,\\u000a\np18\naV# The Southern hemisphere corresponds to zones -1 to -60.\\u000a\np19\naVUTM_PROJECTION_ZONE             = 11\\u000a\np20\naVSUPPRESS_UTM_PROJECTION         = .true.\\u000a\np21\nag6\naV# number of MPI processors\\u000a\np22\naVNPROC                           = 4 \\u000a\np23\nag6\naV# time step parameters\\u000a\np24\naVNSTEP                           = 4096\\u000a\np25\naVDT                              = 0.001\\u000a\np26\nag6\naV#-----------------------------------------------------------\\u000a\np27\naV#\\u000a\np28\naV# LDDRK time scheme\\u000a\np29\naV#\\u000a\np30\naV#-----------------------------------------------------------\\u000a\np31\naVUSE_LDDRK                       = .false.\\u000a\np32\naVINCREASE_CFL_FOR_LDDRK          = .false.\\u000a\np33\naVRATIO_BY_WHICH_TO_INCREASE_IT   = 1.4\\u000a\np34\nag6\naV#-----------------------------------------------------------\\u000a\np35\naV#\\u000a\np36\naV# Mesh\\u000a\np37\naV#\\u000a\np38\naV#-----------------------------------------------------------\\u000a\np39\nag6\naV# Number of nodes for 2D and 3D shape functions for hexahedra.\\u000a\np40\naV# We use either 8-node mesh elements (bricks) or 27-node elements.\\u000a\np41\naV# If you use our internal mesher, the only option is 8-node bricks (27-node elements are not supported).\\u000a\np42\naVNGNOD                           = 8\\u000a\np43\nag6\naV# models:\\u000a\np44\naV# available options are:\\u000a\np45\naV#   default (model parameters described by mesh properties)\\u000a\np46\naV# 1D models available are:\\u000a\np47\naV#   1d_prem,1d_socal,1d_cascadia\\u000a\np48\naV# 3D models available are:\\u000a\np49\naV#   aniso,external,gll,salton_trough,tomo,SEP,coupled,...\\u000a\np50\naVMODEL                           = gll\\u000a\np51\nag6\naV# path for external tomographic models files\\u000a\np52\naVTOMOGRAPHY_PATH                 = ./DATA/tomo_files/\\u000a\np53\naV# if you are using a SEP model (oil-industry format)\\u000a\np54\naVSEP_MODEL_DIRECTORY             = ./DATA/my_SEP_model/\\u000a\np55\nag6\naV#-----------------------------------------------------------\\u000a\np56\nag6\naV# parameters describing the model\\u000a\np57\naVAPPROXIMATE_OCEAN_LOAD          = .false.\\u000a\np58\naVTOPOGRAPHY                      = .false.\\u000a\np59\naVATTENUATION                     = .false.\\u000a\np60\naVANISOTROPY                      = .false.\\u000a\np61\naVGRAVITY                         = .false.\\u000a\np62\nag6\naV# in case of attenuation, reference frequency in Hz at which the velocity values in the velocity model are given (unused otherwise)\\u000a\np63\naVATTENUATION_f0_REFERENCE        = 18.d0\\u000a\np64\nag6\naV# attenuation period range over which we try to mimic a constant Q factor\\u000a\np65\naVMIN_ATTENUATION_PERIOD          = 999999998.d0\\u000a\np66\naVMAX_ATTENUATION_PERIOD          = 999999999.d0\\u000a\np67\naV# ignore this range and ask the code to compute it automatically instead based on the estimated resolution of the mesh (use this unless you know what you are doing)\\u000a\np68\naVCOMPUTE_FREQ_BAND_AUTOMATIC     = .true.\\u000a\np69\nag6\naV# Olsen\'s constant for Q_mu = constant * V_s attenuation rule\\u000a\np70\naVUSE_OLSEN_ATTENUATION           = .false.\\u000a\np71\naVOLSEN_ATTENUATION_RATIO         = 0.05\\u000a\np72\nag6\naV#-----------------------------------------------------------\\u000a\np73\naV#\\u000a\np74\naV# Absorbing boundary conditions\\u000a\np75\naV#\\u000a\np76\naV#-----------------------------------------------------------\\u000a\np77\nag6\naV# C-PML boundary conditions for a regional simulation\\u000a\np78\naV# (if set to .false., and STACEY_ABSORBING_CONDITIONS is also set to .false., you get a free surface instead\\u000a\np79\naV# in the case of elastic or viscoelastic mesh elements, and a rigid surface in the case of acoustic (fluid) elements\\u000a\np80\naVPML_CONDITIONS                  = .false.\\u000a\np81\nag6\naV# C-PML top surface\\u000a\np82\naVPML_INSTEAD_OF_FREE_SURFACE     = .false.\\u000a\np83\nag6\naV# C-PML dominant frequency\\u000a\np84\naVf0_FOR_PML                      = 0.05555\\u000a\np85\nag6\naV# parameters used to rotate C-PML boundary conditions by a given angle (not completed yet)\\u000a\np86\naV# ROTATE_PML_ACTIVATE           = .false.\\u000a\np87\naV# ROTATE_PML_ANGLE              = 0.\\u000a\np88\nag6\naV# absorbing boundary conditions for a regional simulation\\u000a\np89\naV# (if set to .false., and PML_CONDITIONS is also set to .false., you get a free surface instead\\u000a\np90\naV# in the case of elastic or viscoelastic mesh elements, and a rigid surface in the case of acoustic (fluid) elements\\u000a\np91\naVSTACEY_ABSORBING_CONDITIONS     = .true.\\u000a\np92\nag6\naV# absorbing top surface (defined in mesh as \'free_surface_file\')\\u000a\np93\naVSTACEY_INSTEAD_OF_FREE_SURFACE  = .false.\\u000a\np94\nag6\naV# When STACEY_ABSORBING_CONDITIONS is set to .true. :\\u000a\np95\naV# absorbing conditions are defined in xmin, xmax, ymin, ymax and zmin\\u000a\np96\naV# this option BOTTOM_FREE_SURFACE can be set to .true. to\\u000a\np97\naV# make zmin free surface instead of absorbing condition\\u000a\np98\naVBOTTOM_FREE_SURFACE             = .false.\\u000a\np99\nag6\naV#-----------------------------------------------------------\\u000a\np100\naV#\\u000a\np101\naV# undoing attenuation and/or PMLs for sensitivity kernel calculations\\u000a\np102\naV#\\u000a\np103\naV#-----------------------------------------------------------\\u000a\np104\nag6\naV# to undo attenuation and/or PMLs for sensitivity kernel calculations or forward runs with SAVE_FORWARD\\u000a\np105\naV# use the flag below. It performs undoing of attenuation and/or of PMLs in an exact way for sensitivity kernel calculations\\u000a\np106\naV# but requires disk space for temporary storage, and uses a significant amount of memory used as buffers for temporary storage.\\u000a\np107\naV# When that option is on the second parameter indicates how often the code dumps restart files to disk (if in doubt, use something between 100 and 1000).\\u000a\np108\naVUNDO_ATTENUATION_AND_OR_PML     = .false.\\u000a\np109\naVNT_DUMP_ATTENUATION             = 500\\u000a\np110\nag6\naV#-----------------------------------------------------------\\u000a\np111\naV#\\u000a\np112\naV# Visualization\\u000a\np113\naV#\\u000a\np114\naV#-----------------------------------------------------------\\u000a\np115\nag6\naV# save AVS or OpenDX movies\\u000a\np116\naV# MOVIE_TYPE = 1 to show the top surface\\u000a\np117\naV# MOVIE_TYPE = 2 to show all the external faces of the mesh\\u000a\np118\naVCREATE_SHAKEMAP                 = .false.\\u000a\np119\naVMOVIE_SURFACE                   = .false.\\u000a\np120\naVMOVIE_TYPE                      = 2\\u000a\np121\naVMOVIE_VOLUME                    = .false.\\u000a\np122\naVSAVE_DISPLACEMENT               = .true.\\u000a\np123\naVUSE_HIGHRES_FOR_MOVIES          = .false.\\u000a\np124\naVNTSTEP_BETWEEN_FRAMES           = 500\\u000a\np125\naVHDUR_MOVIE                      = 0.0\\u000a\np126\nag6\naV# save AVS or OpenDX mesh files to check the mesh\\u000a\np127\naVSAVE_MESH_FILES                 = .false.\\u000a\np128\nag6\naV# path to store the local database file on each node\\u000a\np129\naVLOCAL_PATH                      = ./OUTPUT_FILES/DATABASES_MPI\\u000a\np130\naV#LOCAL_PATH                      = ./COMMON/DATABASES_MPI\\u000a\np131\nag6\naV# interval at which we output time step info and max of norm of displacement\\u000a\np132\naVNTSTEP_BETWEEN_OUTPUT_INFO      = 1000\\u000a\np133\nag6\naV#-----------------------------------------------------------\\u000a\np134\naV#\\u000a\np135\naV# Sources\\u000a\np136\naV#\\u000a\np137\naV#-----------------------------------------------------------\\u000a\np138\nag6\naV# sources and receivers Z coordinates given directly (i.e. as their true position) instead of as their depth\\u000a\np139\naVUSE_SOURCES_RECEIVERS_Z         = .false.\\u000a\np140\nag6\naV# use a (tilted) FORCESOLUTION force point source (or several) instead of a CMTSOLUTION moment-tensor source.\\u000a\np141\naV# This can be useful e.g. for oil industry foothills simulations or asteroid simulations\\u000a\np142\naV# in which the source is a vertical force, normal force, tilted force, impact etc.\\u000a\np143\naV# If this flag is turned on, the FORCESOLUTION file must be edited by giving:\\u000a\np144\naV# - the corresponding time-shift parameter,\\u000a\np145\naV# - the half duration parameter of the source,\\u000a\np146\naV# - the coordinates of the source,\\u000a\np147\naV# - the magnitude of the force source,\\u000a\np148\naV# - the components of a (non necessarily unitary) direction vector for the force source in the E/N/Z_UP basis.\\u000a\np149\naV# The direction vector is made unitary internally in the code and thus only its direction matters here;\\u000a\np150\naV# its norm is ignored and the norm of the force used is the factor force source times the source time function.\\u000a\np151\naVUSE_FORCE_POINT_SOURCE          = .true.\\u000a\np152\nag6\naV# set to true to use a Ricker source time function instead of the source time functions set by default\\u000a\np153\naV# to represent a (tilted) FORCESOLUTION force point source or a CMTSOLUTION moment-tensor source.\\u000a\np154\naVUSE_RICKER_TIME_FUNCTION        = .false.\\u000a\np155\nag6\naV# Use an external source time function.\\u000a\np156\naV# if .true. you must add a file with your source time function and the file name\\u000a\np157\naV# path relative to lauching directory at the end of CMTSOLUTION or FORCESOURCE file\\u000a\np158\naV# (with multiple sources, one file per source is required).\\u000a\np159\naV# This file must have a single column containing the amplitude of the source at that time step,\\u000a\np160\naV# and on its first line it must contain the time step used, which must be equal to DT as defined at the beginning of this Par_file (the code will check that).\\u000a\np161\naV# Be sure when this option is .false. to remove the name of stf file in CMTSOLUTION or FORCESOURCE\\u000a\np162\naVUSE_EXTERNAL_SOURCE_FILE        = .false.\\u000a\np163\nag6\naV# print source time function\\u000a\np164\naVPRINT_SOURCE_TIME_FUNCTION      = .true.\\u000a\np165\nag6\naV#-----------------------------------------------------------\\u000a\np166\naV#\\u000a\np167\naV# Seismograms\\u000a\np168\naV#\\u000a\np169\naV#-----------------------------------------------------------\\u000a\np170\nag6\naV# interval in time steps for writing of seismograms\\u000a\np171\naVNTSTEP_BETWEEN_OUTPUT_SEISMOS   = 4096\\u000a\np172\nag6\naV# decide if we save displacement, velocity, acceleration and/or pressure in forward runs (they can be set to true simultaneously)\\u000a\np173\naV# currently pressure seismograms are implemented in acoustic (i.e. fluid) elements only\\u000a\np174\naVSAVE_SEISMOGRAMS_DISPLACEMENT   = .true.\\u000a\np175\naVSAVE_SEISMOGRAMS_VELOCITY       = .false.\\u000a\np176\naVSAVE_SEISMOGRAMS_ACCELERATION   = .false.\\u000a\np177\naVSAVE_SEISMOGRAMS_PRESSURE       = .false.   # currently implemented in acoustic (i.e. fluid) elements only\\u000a\np178\nag6\naV# save seismograms also when running the adjoint runs for an inverse problem\\u000a\np179\naV# (usually they are unused and not very meaningful, leave this off in almost all cases)\\u000a\np180\naVSAVE_SEISMOGRAMS_IN_ADJOINT_RUN = .false.\\u000a\np181\nag6\naV# save seismograms in binary or ASCII format (binary is smaller but may not be portable between machines)\\u000a\np182\naVUSE_BINARY_FOR_SEISMOGRAMS      = .false.\\u000a\np183\nag6\naV# output seismograms in Seismic Unix format (binary with 240-byte-headers)\\u000a\np184\naVSU_FORMAT                       = .false.\\u000a\np185\nag6\naV# output seismograms in ASDF (requires asdf-library)\\u000a\np186\naVASDF_FORMAT                     = .false.\\u000a\np187\nag6\naV# decide if master process writes all the seismograms or if all processes do it in parallel\\u000a\np188\naVWRITE_SEISMOGRAMS_BY_MASTER     = .false.\\u000a\np189\nag6\naV# save all seismograms in one large combined file instead of one file per seismogram\\u000a\np190\naV# to avoid overloading shared non-local file systems such as LUSTRE or GPFS for instance\\u000a\np191\naVSAVE_ALL_SEISMOS_IN_ONE_FILE    = .false.\\u000a\np192\nag6\naV# use a trick to increase accuracy of pressure seismograms in fluid (acoustic) elements:\\u000a\np193\naV# use the second derivative of the source for the source time function instead of the source itself,\\u000a\np194\naV# and then record -potential_acoustic() as pressure seismograms instead of -potential_dot_dot_acoustic();\\u000a\np195\naV# this is mathematically equivalent, but numerically significantly more accurate because in the explicit\\u000a\np196\naV# Newmark time scheme acceleration is accurate at zeroth order while displacement is accurate at second order,\\u000a\np197\naV# thus in fluid elements potential_dot_dot_acoustic() is accurate at zeroth order while potential_acoustic()\\u000a\np198\naV# is accurate at second order and thus contains significantly less numerical noise.\\u000a\np199\naVUSE_TRICK_FOR_BETTER_PRESSURE   = .false.\\u000a\np200\nag6\naV#-----------------------------------------------------------\\u000a\np201\naV#\\u000a\np202\naV# Source encoding\\u000a\np203\naV#\\u000a\np204\naV#-----------------------------------------------------------\\u000a\np205\nag6\naV# (for acoustic simulations only for now) determines source encoding factor +/-1 depending on sign of moment tensor\\u000a\np206\naV# (see e.g. Krebs et al., 2009. Fast full-wavefield seismic inversion using encoded sources, Geophysics, 74 (6), WCC177-WCC188.)\\u000a\np207\naVUSE_SOURCE_ENCODING             = .false.\\u000a\np208\nag6\naV#-----------------------------------------------------------\\u000a\np209\naV#\\u000a\np210\naV# Energy calculation\\u000a\np211\naV#\\u000a\np212\naV#-----------------------------------------------------------\\u000a\np213\nag6\naV# to plot energy curves, for instance to monitor how CPML absorbing layers behave;\\u000a\np214\naV# should be turned OFF in most cases because a bit expensive\\u000a\np215\naVOUTPUT_ENERGY                   = .false.\\u000a\np216\naV# every how many time steps we compute energy (which is a bit expensive to compute)\\u000a\np217\naVNTSTEP_BETWEEN_OUTPUT_ENERGY    = 10\\u000a\np218\nag6\naV#-----------------------------------------------------------\\u000a\np219\naV#\\u000a\np220\naV# Adjoint kernel outputs\\u000a\np221\naV#\\u000a\np222\naV#-----------------------------------------------------------\\u000a\np223\nag6\naV# interval in time steps for reading adjoint traces\\u000a\np224\naV# 0 = read the whole adjoint sources at the same time\\u000a\np225\naVNTSTEP_BETWEEN_READ_ADJSRC      = 0\\u000a\np226\nag6\naV# read adjoint sources using ASDF (requires asdf-library)\\u000a\np227\naVREAD_ADJSRC_ASDF                = .false.\\u000a\np228\nag6\naV# this parameter must be set to .true. to compute anisotropic kernels\\u000a\np229\naV# in crust and mantle (related to the 21 Cij in geographical coordinates)\\u000a\np230\naV# default is .false. to compute isotropic kernels (related to alpha and beta)\\u000a\np231\naVANISOTROPIC_KL                  = .false.\\u000a\np232\nag6\naV# compute transverse isotropic kernels (alpha_v,alpha_h,beta_v,beta_h,eta,rho)\\u000a\np233\naV# rather than fully anisotropic kernels in case ANISOTROPIC_KL is set to .true.\\u000a\np234\naVSAVE_TRANSVERSE_KL              = .false.\\u000a\np235\nag6\naV# this parameter must be set to .true. to compute anisotropic kernels for\\u000a\np236\naV# cost function using velocity observable rather than displacement\\u000a\np237\naVANISOTROPIC_VELOCITY_KL         = .false.\\u000a\np238\nag6\naV# outputs approximate Hessian for preconditioning\\u000a\np239\naV#APPROXIMATE_HESS_KL             = .false. ##per Ebru do use for\\u000a\np240\naV#preconditioning\\u000a\np241\naVAPPROXIMATE_HESS_KL             = .true.\\u000a\np242\nag6\naV# save Moho mesh and compute Moho boundary kernels\\u000a\np243\naVSAVE_MOHO_MESH                  = .false.\\u000a\np244\nag6\naV#-----------------------------------------------------------\\u000a\np245\naV#\\u000a\np246\naV# Coupling with an injection technique (DSM, AxiSEM, or FK)\\u000a\np247\naV#\\u000a\np248\naV#-----------------------------------------------------------\\u000a\np249\nag6\naVCOUPLE_WITH_INJECTION_TECHNIQUE = .false.\\u000a\np250\naVINJECTION_TECHNIQUE_TYPE        = 3   # 1 = DSM, 2 = AxiSEM, 3 = FK\\u000a\np251\naVMESH_A_CHUNK_OF_THE_EARTH       = .false.\\u000a\np252\naVTRACTION_PATH                   = ./DATA/AxiSEM_tractions/3/\\u000a\np253\naVFKMODEL_FILE                    = FKmodel\\u000a\np254\naVRECIPROCITY_AND_KH_INTEGRAL     = .false.   # does not work yet\\u000a\np255\nag6\naV#-----------------------------------------------------------\\u000a\np256\nag6\naV# Dimitri Komatitsch, July 2014, CNRS Marseille, France:\\u000a\np257\naV# added the ability to run several calculations (several earthquakes)\\u000a\np258\naV# in an embarrassingly-parallel fashion from within the same run;\\u000a\np259\naV# this can be useful when using a very large supercomputer to compute\\u000a\np260\naV# many earthquakes in a catalog, in which case it can be better from\\u000a\np261\naV# a batch job submission point of view to start fewer and much larger jobs,\\u000a\np262\naV# each of them computing several earthquakes in parallel.\\u000a\np263\naV# To turn that option on, set parameter NUMBER_OF_SIMULTANEOUS_RUNS to a value greater than 1.\\u000a\np264\naV# To implement that, we create NUMBER_OF_SIMULTANEOUS_RUNS MPI sub-communicators,\\u000a\np265\naV# each of them being labeled "my_local_mpi_comm_world", and we use them\\u000a\np266\naV# in all the routines in "src/shared/parallel.f90", except in MPI_ABORT() because in that case\\u000a\np267\naV# we need to kill the entire run.\\u000a\np268\naV# When that option is on, of course the number of processor cores used to start\\u000a\np269\naV# the code in the batch system must be a multiple of NUMBER_OF_SIMULTANEOUS_RUNS,\\u000a\np270\naV# all the individual runs must use the same number of processor cores,\\u000a\np271\naV# which as usual is NPROC in the Par_file,\\u000a\np272\naV# and thus the total number of processor cores to request from the batch system\\u000a\np273\naV# should be NUMBER_OF_SIMULTANEOUS_RUNS * NPROC.\\u000a\np274\naV# All the runs to perform must be placed in directories called run0001, run0002, run0003 and so on\\u000a\np275\naV# (with exactly four digits).\\u000a\np276\naV#\\u000a\np277\naV# Imagine you have 10 independent calculations to do, each of them on 100 cores; you have three options:\\u000a\np278\naV#\\u000a\np279\naV# 1/ submit 10 jobs to the batch system\\u000a\np280\naV#\\u000a\np281\naV# 2/ submit a single job on 1000 cores to the batch, and in that script create a sub-array of jobs to start 10 jobs,\\u000a\np282\naV# each running on 100 cores (see e.g. http://www.schedmd.com/slurmdocs/job_array.html )\\u000a\np283\naV#\\u000a\np284\naV# 3/ submit a single job on 1000 cores to the batch, start SPECFEM3D on 1000 cores, create 10 sub-communicators,\\u000a\np285\naV# cd into one of 10 subdirectories (called e.g. run0001, run0002,... run0010) depending on the sub-communicator\\u000a\np286\naV# your MPI rank belongs to, and run normally on 100 cores using that sub-communicator.\\u000a\np287\naV#\\u000a\np288\naV# The option below implements 3/.\\u000a\np289\naV#\\u000a\np290\naVNUMBER_OF_SIMULTANEOUS_RUNS     = 1\\u000a\np291\nag6\naV# if we perform simultaneous runs in parallel, if only the source and receivers vary between these runs\\u000a\np292\naV# but not the mesh nor the model (velocity and density) then we can also read the mesh and model files\\u000a\np293\naV# from a single run in the beginning and broadcast them to all the others; for a large number of simultaneous\\u000a\np294\naV# runs for instance when solving inverse problems iteratively this can DRASTICALLY reduce I/Os to disk in the solver\\u000a\np295\naV# (by a factor equal to NUMBER_OF_SIMULTANEOUS_RUNS), and reducing I/Os is crucial in the case of huge runs.\\u000a\np296\naV# Thus, always set this option to .true. if the mesh and the model are the same for all simultaneous runs.\\u000a\np297\naV# In that case there is no need to duplicate the mesh and model file database (the content of the DATABASES_MPI\\u000a\np298\naV# directories) in each of the run0001, run0002,... directories, it is sufficient to have one in run0001\\u000a\np299\naV# and the code will broadcast it to the others)\\u000a\np300\naVBROADCAST_SAME_MESH_AND_MODEL   = .true.\\u000a\np301\nag6\naV#-----------------------------------------------------------\\u000a\np302\nag6\naV# set to true to use GPUs\\u000a\np303\naVGPU_MODE                        = .true.\\u000a\np304\nag6\naV# ADIOS Options for I/Os\\u000a\np305\naVADIOS_ENABLED                   = .false.\\u000a\np306\naVADIOS_FOR_DATABASES             = .false.\\u000a\np307\naVADIOS_FOR_MESH                  = .false.\\u000a\np308\naVADIOS_FOR_FORWARD_ARRAYS        = .false.\\u000a\np309\naVADIOS_FOR_KERNELS               = .false.\\u000a\np310\nag6\naV#**********************\\u000a\np311\naV#Serial mesh decomposer\\u000a\np312\naV#**********************\\u000a\np313\nag6\naVLTS_MODE                        = .false.\\u000a\np314\nag6\naVPARTITIONING_TYPE               = 1\\u000a\np315\na."""

In [ ]:
print(test_parfile_str == parfile_pickle_str)
#print(set(test_parfile_str.split()).difference(set(parfile_pickle_str.split())))

In [ ]:
x = goobleydoobleydoo('42')

In [ ]:
def sillyfunc(index=None):
    if index == None:
        print('none')
    else:
        print(f'index={index}')
        
sillyfunc(2)

In [ ]:
df_rec = []
for i in range(6):
    df_rec.append({'sid':i%3,'eid':0})
df = pd.DataFrame.from_records(df_rec)
print(df)
print()

print(df['sid'])
print()
s = set(df['sid'])
ss = set(df['sid'])
print(s)
print(f's == ss: {s == ss}')

In [ ]:
tmp_fqp = os.path.join(data_out_dir,'tmp')
!ls {tmp_fqp}

In [ ]:
import os
from glob import glob
for src_path in glob(data_out_dir):
    print(src_path)
    print(os.path.relpath(src_path, 'tmp'))
    '''
    os.symlink(
        os.path.relpath(
            src_path,
            'dst/'
        ),
        os.path.join('dst', os.path.basename(src_path))
    )
    '''

In [ ]:
#p = os.path.relpath('tmp', start=os.curdir)
p = os.path.relpath(tmp_fqp, start='data/')
fqpname = os.path.join(p, 'tmp')

print(p)
print(fqpname)

In [ ]:
tpath = 'data/output/tmp/TestProjects/FirstTestProject/run0001/DATA'
os.symlink('STAIONS.sid0','data/output/tmp/TestProjects/FirstTestProject/run0001/DATA/LINKSTATIONS')
!ls -ltrh {tpath}

In [ ]:
test_mesh_root = '/Users/seismac/Documents/Work/Bench/ForkGnam/pyaspect/notebooks/data/output/tmp/TestProjects/'
test_mesh_name = 'MESH-default_norot_xsft4400_ysft19100_quart_100m_min_vs_sig250m'
test_src = os.path.join(test_mesh_root,test_mesh_name)
test_dst = os.path.join(test_mesh_root,'copy_MESH-default')
print(test_src)
print(test_dst)

In [ ]:
import shutil

try:
    shutil.copytree(test_src, test_dst)
except Exception as e:
    print(e)

In [ ]:
!ls -dltrh {test_src}
!ls -dltrh {test_dst}

In [ ]:
#!ls {data_out_dir}
ztr = np.fromfile(data_out_dir + 's00.t000960g03.FXZ.semd',dtype=np.float32)

plt.plot(ztr[1243:1260])

In [ ]:
np.savez_compressed(data_out_dir + 's00.t000960g03.FXZ.npz',ztr=ztr)

In [ ]:
start_dir = os.path.join(data_out_dir,'SYN')
'''
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print os.path.join(subdir, file)
''';
        
all_traces = []
i = 0
for entry in os.scandir(start_dir):
    #if entry.path.endswith('FXZ.semd'):
    if entry.path.endswith('.semd'):
        all_traces.append(np.fromfile(entry.path,dtype=np.float32))
    if i == 4:
        break

stk_traces = np.stack(all_traces,axis=0)
print(stk_traces.shape)
print(stk_traces.dtype)
print(stk_traces.nbytes/(1024**2))

out_comp_file = os.path.join(start_dir , 'stk_traces_compressed.npz')
np.savez_compressed(out_comp_file,stk_traces=stk_traces)
!ls -ltrh {out_comp_file}

In [ ]:
!ls /Users/seismac/Seismic/

In [ ]:
xran = np.random.randn(3, 4)
print('xran:\n',xran.shape)
arrays = [np.random.randn(3, 4) for _ in range(10)]
print('arrays:\n',arrays)

stk = np.stack(arrays, axis=0)
print('stk:\n',stk)